<a href="https://colab.research.google.com/github/areeba42197/Cryptography-Tool/blob/main/cryptographytool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile cipher_functions.py

def additive_cipher_encrypt(plaintext, key):
    ciphertext = ''
    for char in plaintext:
        if char.isalpha():  # Check if it's an alphabetic character
            shift = (ord(char.lower()) - ord('a') + key) % 26
            new_char = chr(shift + ord('a'))
            ciphertext += new_char.upper() if char.isupper() else new_char
        else:
            ciphertext += char  # Non-alphabetic characters remain the same
    return ciphertext

def additive_cipher_decrypt(ciphertext, key):
    return additive_cipher_encrypt(ciphertext, -key)  # Decryption is reverse shift

from math import gcd

def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def multiplicative_cipher_encrypt(plaintext, key):
    if gcd(key, 26) != 1:
        raise ValueError("Key must be coprime with 26")

    ciphertext = ''
    for char in plaintext:
        if char.isalpha():
            new_pos = (ord(char.lower()) - ord('a')) * key % 26
            new_char = chr(new_pos + ord('a'))
            ciphertext += new_char.upper() if char.isupper() else new_char
        else:
            ciphertext += char
    return ciphertext

def multiplicative_cipher_decrypt(ciphertext, key):
    inverse_key = mod_inverse(key, 26)
    if inverse_key is None:
        raise ValueError("No multiplicative inverse for the given key")
    return multiplicative_cipher_encrypt(ciphertext, inverse_key)

def affine_cipher_encrypt(plaintext, a, b):
    if gcd(a, 26) != 1:
        raise ValueError("Key 'a' must be coprime with 26")

    ciphertext = ''
    for char in plaintext:
        if char.isalpha():
            new_pos = ((ord(char.lower()) - ord('a')) * a + b) % 26
            new_char = chr(new_pos + ord('a'))
            ciphertext += new_char.upper() if char.isupper() else new_char
        else:
            ciphertext += char
    return ciphertext

def affine_cipher_decrypt(ciphertext, a, b):
    inverse_a = mod_inverse(a, 26)
    if inverse_a is None:
        raise ValueError("No multiplicative inverse for the given key 'a'")

    plaintext = ''
    for char in ciphertext:
        if char.isalpha():
            new_pos = (inverse_a * ((ord(char.lower()) - ord('a')) - b)) % 26
            new_char = chr(new_pos + ord('a'))
            plaintext += new_char.upper() if char.isupper() else new_char
        else:
            plaintext += char
    return plaintext

# Encryption function for Monoalphabetic Cipher
def monoalphabetic_cipher_encrypt(plaintext, substitution_key):
    # Check if the substitution key is exactly 26 characters and only contains alphabetic characters
    if len(substitution_key) != 26 or not substitution_key.isalpha():
        raise ValueError("Substitution key must be exactly 26 alphabetic characters.")

    # Convert the substitution key to lowercase for uniformity
    substitution_key = substitution_key.lower()

    # Define the alphabet
    alphabet = 'abcdefghijklmnopqrstuvwxyz'

    # Create a cipher map where each letter in the alphabet maps to a letter in the substitution key
    cipher_map = {alphabet[i]: substitution_key[i] for i in range(26)}

    # Encrypt the plaintext
    ciphertext = ''
    for char in plaintext:
        if char.isalpha():
            # Encrypt the character by substituting it using the cipher map
            new_char = cipher_map[char.lower()]
            ciphertext += new_char.upper() if char.isupper() else new_char
        else:
            # Keep non-alphabetic characters unchanged
            ciphertext += char

    return ciphertext


# Decryption function for Monoalphabetic Cipher
def monoalphabetic_cipher_decrypt(ciphertext, substitution_key):
    # Check if the substitution key is exactly 26 characters and only contains alphabetic characters
    if len(substitution_key) != 26 or not substitution_key.isalpha():
        raise ValueError("Substitution key must be exactly 26 alphabetic characters.")

    # Convert the substitution key to lowercase for uniformity
    substitution_key = substitution_key.lower()

    # Define the alphabet
    alphabet = 'abcdefghijklmnopqrstuvwxyz'

    # Create a reverse cipher map to map substituted characters back to the original alphabet
    reverse_map = {substitution_key[i]: alphabet[i] for i in range(26)}

    # Decrypt the ciphertext
    plaintext = ''
    for char in ciphertext:
        if char.isalpha():
            # Decrypt the character by substituting it using the reverse cipher map
            new_char = reverse_map[char.lower()]
            plaintext += new_char.upper() if char.isupper() else new_char
        else:
            # Keep non-alphabetic characters unchanged
            plaintext += char

    return plaintext



def autokey_cipher_encrypt(plaintext, keyword):
    keyword = (keyword + plaintext).lower()[:len(plaintext)]  # Extend keyword with plaintext
    ciphertext = ''
    for i, char in enumerate(plaintext):
        if char.isalpha():
            shift = (ord(char.lower()) - ord('a') + ord(keyword[i]) - ord('a')) % 26
            new_char = chr(shift + ord('a'))
            ciphertext += new_char.upper() if char.isupper() else new_char
        else:
            ciphertext += char
    return ciphertext

def autokey_cipher_decrypt(ciphertext, keyword):
    keyword = keyword.lower()
    plaintext = ''
    for i, char in enumerate(ciphertext):
        if char.isalpha():
            shift = (ord(char.lower()) - ord(keyword[i]) + 26) % 26
            new_char = chr(shift + ord('a'))
            plaintext += new_char.upper() if char.isupper() else new_char
            keyword += new_char  # Add plaintext letter to the keyword for future decryption
        else:
            plaintext += char
    return plaintext

def generate_playfair_matrix(keyword):
    keyword = keyword.lower().replace('j', 'i')
    alphabet = 'abcdefghiklmnopqrstuvwxyz'  # No 'j'
    matrix = []
    used_chars = set()

    for char in keyword + alphabet:
        if char not in used_chars:
            matrix.append(char)
            used_chars.add(char)

    return [matrix[i:i+5] for i in range(0, 25, 5)]

def find_position(matrix, char):
    for row in range(5):
        for col in range(5):
            if matrix[row][col] == char:
                return row, col
    return None

def playfair_cipher_encrypt(plaintext, keyword):
    matrix = generate_playfair_matrix(keyword)
    plaintext = plaintext.lower().replace('j', 'i')
    plaintext_pairs = []

    # Prepare digraphs
    i = 0
    while i < len(plaintext):
        if i == len(plaintext) - 1:
            plaintext_pairs.append(plaintext[i] + 'x')
            i += 1
        elif plaintext[i] == plaintext[i+1]:
            plaintext_pairs.append(plaintext[i] + 'x')
            i += 1
        else:
            plaintext_pairs.append(plaintext[i] + plaintext[i+1])
            i += 2

    # Encrypt each digraph
    ciphertext = ''
    for pair in plaintext_pairs:
        row1, col1 = find_position(matrix, pair[0])
        row2, col2 = find_position(matrix, pair[1])

        if row1 == row2:
            ciphertext += matrix[row1][(col1 + 1) % 5] + matrix[row2][(col2 + 1) % 5]
        elif col1 == col2:
            ciphertext += matrix[(row1 + 1) % 5][col1] + matrix[(row2 + 1) % 5][col2]
        else:
            ciphertext += matrix[row1][col2] + matrix[row2][col1]

    return ciphertext

def playfair_cipher_decrypt(ciphertext, keyword):
    matrix = generate_playfair_matrix(keyword)
    plaintext = ''

    # Decrypt each digraph
    for i in range(0, len(ciphertext), 2):
        row1, col1 = find_position(matrix, ciphertext[i])
        row2, col2 = find_position(matrix, ciphertext[i+1])

        if row1 == row2:
            plaintext += matrix[row1][(col1 - 1) % 5] + matrix[row2][(col2 - 1) % 5]
        elif col1 == col2:
            plaintext += matrix[(row1 - 1) % 5][col1] + matrix[(row2 - 1) % 5][col2]
        else:
            plaintext += matrix[row1][col2] + matrix[row2][col1]

    return plaintext


def vigenere_cipher_encrypt(plaintext, keyword):
    keyword = keyword.lower()
    keyword_repeated = (keyword * (len(plaintext) // len(keyword) + 1))[:len(plaintext)]

    ciphertext = ''
    for i, char in enumerate(plaintext):
        if char.isalpha():
            shift = (ord(char.lower()) - ord('a') + ord(keyword_repeated[i]) - ord('a')) % 26
            new_char = chr(shift + ord('a'))
            ciphertext += new_char.upper() if char.isupper() else new_char
        else:
            ciphertext += char
    return ciphertext

def vigenere_cipher_decrypt(ciphertext, keyword):
    keyword = keyword.lower()
    keyword_repeated = (keyword * (len(ciphertext) // len(keyword) + 1))[:len(ciphertext)]

    plaintext = ''
    for i, char in enumerate(ciphertext):
        if char.isalpha():
            shift = (ord(char.lower()) - ord(keyword_repeated[i]) + 26) % 26
            new_char = chr(shift + ord('a'))
            plaintext += new_char.upper() if char.isupper() else new_char
        else:
            plaintext += char
    return plaintext

Writing cipher_functions.py


In [ ]:
!pip install streamlit
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
%%writefile streamlit_app.py
import streamlit as st

# Import all the functions from cipher_functions.py
from cipher_functions import (
    additive_cipher_encrypt, additive_cipher_decrypt,
    multiplicative_cipher_encrypt, multiplicative_cipher_decrypt,
    affine_cipher_encrypt, affine_cipher_decrypt,
    monoalphabetic_cipher_encrypt, monoalphabetic_cipher_decrypt,
    autokey_cipher_encrypt, autokey_cipher_decrypt,
    playfair_cipher_encrypt, playfair_cipher_decrypt,
    vigenere_cipher_encrypt, vigenere_cipher_decrypt
)

def main():
    # Title with dark blue color and centered
    st.markdown("<h1 style='text-align: center; color: #003366;'>Cryptography Lab</h1>", unsafe_allow_html=True)

    st.markdown("<div style='height: 30px;'></div>", unsafe_allow_html=True)

    # Introduction with a clean, concise format
    st.markdown("""
    Welcome to *The Cryptography Lab*! Here, you can explore various encryption and decryption ciphers in real-time.
    Select a cipher, input your text, and provide the necessary keys to perform encryption or decryption.
    """)

    # Step 1: Select Cipher
    st.subheader("Choose Your Cipher")
    cipher = st.selectbox("Pick a Cipher", [
        'Additive', 'Multiplicative', 'Affine', 'Monoalphabetic', 'Autokey',
        'Playfair', 'Vigenère'
    ], key="cipher")

    # Step 2: Select Mode (Encrypt/Decrypt)
    st.subheader("Choose Mode")
    mode = st.radio("Select Operation Mode", ('Encrypt', 'Decrypt'), key="mode")

    # Step 3: Input Text
    st.subheader("Enter Your Text")
    text = st.text_area("Enter Plaintext (for encryption) or Ciphertext (for decryption)", key="text")

    # Add some helpful instructions for the user
    st.markdown("""
    *Tips:*
    - For encryption, input the plain text message.
    - For decryption, input the encrypted message.
    """)

    # Step 4: Key Input
    st.subheader("Enter Keys")
    key1 = st.text_input("Enter Key 1 (required for most ciphers)", key="key1")
    key2 = st.text_input("Enter Key 2 (only for certain ciphers like Affine, optional)", key="key2")

    # Step 5: Encryption/Decryption Button with interactive effects
    if st.button(f"Perform {mode}"):
        if not text:
            st.error("Oops! Please enter some text to encrypt or decrypt.")
        else:
            result = ""
            try:
                # Handling cipher encryption and decryption logic
                if cipher == 'Additive':
                    if not key1.isdigit():
                        st.error("Additive Cipher requires a numeric key.")
                    else:
                        key = int(key1)
                        result = additive_cipher_encrypt(text, key) if mode == 'Encrypt' else additive_cipher_decrypt(text, key)

                elif cipher == 'Multiplicative':
                    if not key1.isdigit():
                        st.error("Multiplicative Cipher requires a numeric key.")
                    else:
                        key = int(key1)
                        result = multiplicative_cipher_encrypt(text, key) if mode == 'Encrypt' else multiplicative_cipher_decrypt(text, key)

                elif cipher == 'Affine':
                    if not key1.isdigit() or not key2.isdigit():
                        st.error("Affine Cipher requires two numeric keys.")
                    else:
                        a, b = int(key1), int(key2)
                        result = affine_cipher_encrypt(text, a, b) if mode == 'Encrypt' else affine_cipher_decrypt(text, a, b)

                elif cipher == 'Monoalphabetic':
                    if not key1:
                        st.error("Monoalphabetic Cipher requires a substitution key.")
                    else:
                        result = monoalphabetic_cipher_encrypt(text, key1) if mode == 'Encrypt' else monoalphabetic_cipher_decrypt(text, key1)

                elif cipher == 'Autokey':
                    if not key1:
                        st.error("Autokey Cipher requires a keyword.")
                    else:
                        result = autokey_cipher_encrypt(text, key1) if mode == 'Encrypt' else autokey_cipher_decrypt(text, key1)

                elif cipher == 'Playfair':
                    if not key1:
                        st.error("Playfair Cipher requires a keyword.")
                    else:
                        result = playfair_cipher_encrypt(text, key1) if mode == 'Encrypt' else playfair_cipher_decrypt(text, key1)

                elif cipher == 'Vigenère':
                    if not key1:
                        st.error("Vigenère Cipher requires a keyword.")
                    else:
                        result = vigenere_cipher_encrypt(text, key1) if mode == 'Encrypt' else vigenere_cipher_decrypt(text, key1)

                # Display the result
                if result:
                    st.success(f"Success! Here's the {mode}ed text: \n\n{result}")

            except Exception as e:
                st.error(f"An error occurred: {e}")

    # Interactive help button for additional user support
    if st.button("Need Help?"):
        st.info("""
        *Help with Ciphers:*
        - *Additive Cipher*: A simple cipher where each letter in the plaintext is shifted by a certain number.
        - *Multiplicative Cipher*: A cipher that shifts letters based on a multiplier.
        - *Affine Cipher*: Combines additive and multiplicative shifts.
        - *Monoalphabetic Cipher*: A cipher where each letter is substituted with another letter in a fixed manner.
        - *Autokey Cipher*: Uses a keyword to modify the plaintext during encryption.
        - *Playfair Cipher*: A digraph cipher that encrypts pairs of letters.
        - *Vigenère Cipher*: A polyalphabetic cipher that uses a keyword to shift letters based on their position.

        Check the *Instructions* section for tips on how to use these ciphers.
        """)

    # A brief footer for extra charm
    st.markdown("""
    Developed with love by Cryptography Enthusiasts.
    Feel free to try multiple ciphers and experiment with different keys. Happy encrypting!
    """)

if __name__ == "__main__":
    main()

Overwriting streamlit_app.py


In [ ]:
from pyngrok import ngrok

# Replace with your actual ngrok authtoken
ngrok.set_auth_token("2n9bZmnMdMY33xHkCV8mRQdltsV_2FqvVB7kbwaAiUWHFTZMM")

# Create the ngrok tunnel
public_url = ngrok.connect(8501)
print(f" * Streamlit app running at: {public_url}")

# Run the Streamlit app
!streamlit run streamlit_app.py &

 * Streamlit app running at: NgrokTunnel: "https://91dd-34-74-151-96.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.151.96:8501

